In [ ]:
# Import Required Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.optimize import minimize
import warnings
warnings.filterwarnings('ignore')

# Time series and volatility modeling
from statsmodels.tsa.arima.model import ARIMA
from arch import arch_model
from statsmodels.stats.diagnostic import acorr_ljungbox
from statsmodels.tsa.stattools import adfuller

# Risk calculations
from scipy.stats import norm

# Set random seed for reproducibility
np.random.seed(42)

# Configure plotting
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 10

print("All libraries imported successfully")
print("Plotting configuration set")
print("Random seed set for reproducibility")


In [ ]:
# Define simulation parameters
SIMULATION_DAYS = 180
START_DATE = pd.Timestamp('2024-01-01')

# Currency parameters (based on typical market characteristics)
currency_params = {
    'USD': {'initial_rate': 1.0000, 'annual_vol': 0.00, 'drift': 0.0000},  # Base currency
    'EUR': {'initial_rate': 1.0850, 'annual_vol': 0.12, 'drift': -0.0010},  # EUR/USD
    'GBP': {'initial_rate': 1.2650, 'annual_vol': 0.14, 'drift': -0.0015},  # GBP/USD
    'JPY': {'initial_rate': 0.0067, 'annual_vol': 0.11, 'drift': -0.0005},  # USD/JPY inverted
    'EM_BASKET': {'initial_rate': 0.8500, 'annual_vol': 0.18, 'drift': -0.0020}  # EM/USD
}

def generate_fx_rates(currencies, params, days, start_date):
    """
    Generate synthetic FX rates using geometric Brownian motion
    """
    dates = pd.date_range(start=start_date, periods=days, freq='D')
    fx_data = pd.DataFrame(index=dates)
    
    dt = 1/252  # Daily time step (252 trading days per year)
    
    for currency in currencies:
        if currency == 'USD':
            # USD is base currency
            fx_data[currency] = 1.0
            continue
            
        # Extract parameters
        S0 = params[currency]['initial_rate']
        sigma = params[currency]['annual_vol']
        mu = params[currency]['drift']
        
        # Generate random shocks
        np.random.seed(42 + hash(currency) % 100)  # Different seed per currency
        dW = np.random.normal(0, np.sqrt(dt), days)
        
        # Initialize rate series
        rates = np.zeros(days)
        rates[0] = S0
        
        # Generate rates using geometric Brownian motion
        for t in range(1, days):
            rates[t] = rates[t-1] * np.exp((mu - 0.5 * sigma**2) * dt + sigma * dW[t])
        
        fx_data[currency] = rates
    
    return fx_data

# Generate FX rates
currencies = ['USD', 'EUR', 'GBP', 'JPY', 'EM_BASKET']
fx_rates = generate_fx_rates(currencies, currency_params, SIMULATION_DAYS, START_DATE)

# Calculate daily returns
fx_returns = fx_rates.pct_change().dropna()

# Display basic statistics
print("FX Rate Generation Complete")
print("\nInitial FX Rates:")
print(fx_rates.iloc[0])
print("\nFinal FX Rates:")
print(fx_rates.iloc[-1])
print("\nDaily Return Statistics:")
print(fx_returns.describe())


In [ ]:
# Portfolio configuration
INITIAL_PORTFOLIO_VALUE = 100_000_000  # $100M portfolio

# Realistic currency exposure weights (based on typical global portfolio)
portfolio_weights = {
    'USD': 0.45,      # 45% US exposure
    'EUR': 0.25,      # 25% European exposure  
    'GBP': 0.10,      # 10% UK exposure
    'JPY': 0.12,      # 12% Japanese exposure
    'EM_BASKET': 0.08 # 8% Emerging markets exposure
}

# Verify weights sum to 1
assert abs(sum(portfolio_weights.values()) - 1.0) < 1e-10

def generate_local_asset_returns(currencies, days, start_date):
    """
    Generate synthetic local currency asset returns (equity/bond returns before FX)
    """
    dates = pd.date_range(start=start_date, periods=days, freq='D')[1:]  # Exclude first day
    local_returns = pd.DataFrame(index=dates)
    
    # Asset return parameters by region (annual figures)
    asset_params = {
        'USD': {'mean': 0.08, 'vol': 0.16},      # US equity-like returns
        'EUR': {'mean': 0.06, 'vol': 0.18},      # European equity
        'GBP': {'mean': 0.07, 'vol': 0.19},      # UK equity
        'JPY': {'mean': 0.04, 'vol': 0.20},      # Japanese equity
        'EM_BASKET': {'mean': 0.09, 'vol': 0.25} # EM equity (higher vol, higher return)
    }
    
    dt = 1/252  # Daily time step
    
    for currency in currencies:
        np.random.seed(123 + hash(currency) % 100)  # Different seed for asset returns
        
        mu = asset_params[currency]['mean']
        sigma = asset_params[currency]['vol']
        
        # Generate daily returns (normal distribution)
        daily_returns = np.random.normal(mu * dt, sigma * np.sqrt(dt), days - 1)
        local_returns[f'{currency}_local'] = daily_returns
    
    return local_returns

def calculate_unhedged_portfolio_returns(fx_returns, local_returns, weights):
    """
    Calculate total portfolio returns including FX effects (unhedged)
    Total return = Local asset return + FX return + Cross-effect
    """
    portfolio_returns = pd.DataFrame(index=fx_returns.index)
    
    total_return = 0
    for currency in weights.keys():
        weight = weights[currency]
        
        if currency == 'USD':
            # USD assets have no FX effect
            currency_return = local_returns[f'{currency}_local'] * weight
        else:
            # Non-USD assets have FX translation effect
            fx_return = fx_returns[currency]
            local_return = local_returns[f'{currency}_local']
            
            # Total return = (1 + local) * (1 + fx) - 1 ≈ local + fx + local*fx
            total_currency_return = local_return + fx_return + (local_return * fx_return)
            currency_return = total_currency_return * weight
        
        portfolio_returns[f'{currency}_contribution'] = currency_return
        total_return += currency_return
    
    portfolio_returns['total_return'] = total_return
    return portfolio_returns

# Generate local asset returns
local_asset_returns = generate_local_asset_returns(currencies, SIMULATION_DAYS, START_DATE)

# Calculate unhedged portfolio returns
unhedged_returns = calculate_unhedged_portfolio_returns(fx_returns, local_asset_returns, portfolio_weights)

# Calculate portfolio value evolution
portfolio_values = pd.DataFrame(index=fx_rates.index)
portfolio_values.iloc[0] = INITIAL_PORTFOLIO_VALUE

for i in range(1, len(portfolio_values)):
    portfolio_values.iloc[i] = portfolio_values.iloc[i-1] * (1 + unhedged_returns['total_return'].iloc[i-1])

portfolio_values.columns = ['unhedged_value']

print("Portfolio Setup Complete")
print(f"\nPortfolio Weights:")
for currency, weight in portfolio_weights.items():
    print(f"  {currency}: {weight:.1%}")

print(f"\nInitial Portfolio Value: ${INITIAL_PORTFOLIO_VALUE:,.0f}")
print(f"Final Unhedged Value: ${portfolio_values.iloc[-1].values[0]:,.0f}")
print(f"Total Unhedged Return: {(portfolio_values.iloc[-1].values[0]/INITIAL_PORTFOLIO_VALUE - 1):.2%}")

print(f"\nDaily Return Statistics (Unhedged):")
print(unhedged_returns['total_return'].describe())


In [ ]:
def calculate_hedged_portfolio_returns(fx_returns, local_returns, weights, hedge_ratio):
    """
    Calculate portfolio returns with specified hedge ratio
    
    hedge_ratio: 0.0 = no hedging, 1.0 = full hedging
    Hedging reduces FX impact proportionally to hedge ratio
    """
    portfolio_returns = pd.DataFrame(index=fx_returns.index)
    
    total_return = 0
    for currency in weights.keys():
        weight = weights[currency]
        
        if currency == 'USD':
            # USD assets have no FX effect regardless of hedging
            currency_return = local_returns[f'{currency}_local'] * weight
        else:
            # Non-USD assets: apply hedge ratio to FX component
            fx_return = fx_returns[currency]
            local_return = local_returns[f'{currency}_local']
            
            # Hedged FX return = (1 - hedge_ratio) * fx_return
            hedged_fx_return = (1 - hedge_ratio) * fx_return
            
            # Total return with hedging
            total_currency_return = local_return + hedged_fx_return + (local_return * hedged_fx_return)
            currency_return = total_currency_return * weight
        
        portfolio_returns[f'{currency}_contribution'] = currency_return
        total_return += currency_return
    
    portfolio_returns['total_return'] = total_return
    return portfolio_returns

def calculate_hedging_cost(hedge_ratio, notional_exposure):
    """
    Simplified hedging cost calculation
    Assumes 0.1% annual cost for full hedging, scaled by hedge ratio and exposure
    """
    annual_cost_rate = 0.001  # 0.1% annual
    daily_cost_rate = annual_cost_rate / 252
    
    # Cost proportional to hedge ratio and non-USD exposure
    non_usd_weight = 1 - portfolio_weights['USD']  # 55% non-USD exposure
    daily_cost = hedge_ratio * non_usd_weight * daily_cost_rate
    
    return daily_cost

# Define hedge ratios to analyze
hedge_ratios = [0.0, 0.5, 1.0]
hedge_labels = ['Unhedged (0%)', 'Partial Hedge (50%)', 'Full Hedge (100%)']

# Calculate returns for each hedge ratio
hedged_results = {}
portfolio_values_all = pd.DataFrame(index=fx_rates.index)
portfolio_values_all['unhedged_value'] = portfolio_values['unhedged_value']

for i, hedge_ratio in enumerate(hedge_ratios):
    label = hedge_labels[i]
    
    if hedge_ratio == 0.0:
        # Use existing unhedged results
        hedged_results[label] = unhedged_returns.copy()
    else:
        # Calculate hedged returns
        hedged_returns = calculate_hedged_portfolio_returns(
            fx_returns, local_asset_returns, portfolio_weights, hedge_ratio
        )
        
        # Subtract hedging costs
        hedging_cost = calculate_hedging_cost(hedge_ratio, INITIAL_PORTFOLIO_VALUE)
        hedged_returns['total_return'] -= hedging_cost
        
        hedged_results[label] = hedged_returns
    
    # Calculate portfolio value evolution for this hedge ratio
    values = pd.Series(index=fx_rates.index, dtype=float)
    values.iloc[0] = INITIAL_PORTFOLIO_VALUE
    
    for j in range(1, len(values)):
        values.iloc[j] = values.iloc[j-1] * (1 + hedged_results[label]['total_return'].iloc[j-1])
    
    portfolio_values_all[f'{label.lower().replace(" ", "_").replace("(", "").replace(")", "").replace("%", "pct")}_value'] = values

# Performance summary
print("Hedge Ratio Analysis Complete")
print("\nPerformance Summary (180-day period):")
print("=" * 60)

for i, hedge_ratio in enumerate(hedge_ratios):
    label = hedge_labels[i]
    col_name = f'{label.lower().replace(" ", "_").replace("(", "").replace(")", "").replace("%", "pct")}_value'
    
    initial_val = portfolio_values_all[col_name].iloc[0]
    final_val = portfolio_values_all[col_name].iloc[-1]
    total_return = (final_val / initial_val - 1)
    
    returns_series = hedged_results[label]['total_return']
    volatility = returns_series.std() * np.sqrt(252)  # Annualized
    sharpe = (returns_series.mean() * 252) / volatility if volatility > 0 else 0
    
    print(f"\n{label}:")
    print(f"  Final Value: ${final_val:,.0f}")
    print(f"  Total Return: {total_return:.2%}")
    print(f"  Annualized Vol: {volatility:.2%}")
    print(f"  Sharpe Ratio: {sharpe:.3f}")

# Calculate correlation matrix of hedge strategies
hedge_returns_df = pd.DataFrame()
for i, hedge_ratio in enumerate(hedge_ratios):
    label = hedge_labels[i]
    hedge_returns_df[label] = hedged_results[label]['total_return']

print(f"\nCorrelation Matrix:")
print(hedge_returns_df.corr().round(3))


In [ ]:
def fit_arima_garch_model(returns_series, currency_name):
    """
    Fit ARIMA-GARCH model to currency returns
    """
    try:
        # Remove any infinite or NaN values
        returns_clean = returns_series.dropna()
        returns_clean = returns_clean[np.isfinite(returns_clean)]
        
        if len(returns_clean) < 50:  # Need minimum data
            return None, None, None
        
        # Convert to percentage for numerical stability
        returns_pct = returns_clean * 100
        
        # Step 1: Fit ARIMA model for mean equation
        # Use simple ARIMA(1,0,1) for mean
        try:
            arima_model = ARIMA(returns_pct, order=(1, 0, 1))
            arima_fit = arima_model.fit()
            arima_residuals = arima_fit.resid
        except:
            # Fallback to simpler model if convergence issues
            arima_model = ARIMA(returns_pct, order=(0, 0, 0))
            arima_fit = arima_model.fit()
            arima_residuals = arima_fit.resid
        
        # Step 2: Fit GARCH model for volatility
        # Use GARCH(1,1) which is standard in finance
        try:
            garch_model = arch_model(arima_residuals, vol='GARCH', p=1, q=1, rescale=False)
            garch_fit = garch_model.fit(disp='off')
            
            # Generate volatility forecasts
            forecast_horizon = 30  # 30-day forecast
            forecasts = garch_fit.forecast(horizon=forecast_horizon)
            vol_forecast = np.sqrt(forecasts.variance.values[-1, :])  # Convert to volatility
            
        except:
            # Fallback: use rolling volatility if GARCH fails
            vol_forecast = np.full(30, returns_clean.std() * 100)
            garch_fit = None
        
        return arima_fit, garch_fit, vol_forecast / 100  # Convert back from percentage
        
    except Exception as e:
        print(f"Warning: Model fitting failed for {currency_name}: {str(e)}")
        return None, None, None

def analyze_volatility_patterns(fx_returns):
    """
    Analyze volatility patterns and clustering
    """
    vol_analysis = {}
    
    for currency in fx_returns.columns:
        if currency == 'USD':
            continue  # Skip USD (base currency)
            
        returns = fx_returns[currency]
        
        # Calculate rolling volatility (21-day window)
        rolling_vol = returns.rolling(window=21).std() * np.sqrt(252)  # Annualized
        
        # Volatility clustering test (Ljung-Box on squared returns)
        try:
            lb_test = acorr_ljungbox(returns**2, lags=10, return_df=True)
            volatility_clustering = lb_test['lb_pvalue'].iloc[4] < 0.05  # 5-day lag
        except:
            volatility_clustering = False
        
        # Fit ARIMA-GARCH
        arima_fit, garch_fit, vol_forecast = fit_arima_garch_model(returns, currency)
        
        vol_analysis[currency] = {
            'rolling_vol': rolling_vol,
            'vol_clustering': volatility_clustering,
            'arima_model': arima_fit,
            'garch_model': garch_fit,
            'vol_forecast': vol_forecast,
            'current_vol': rolling_vol.iloc[-1] if not pd.isna(rolling_vol.iloc[-1]) else returns.std() * np.sqrt(252)
        }
    
    return vol_analysis

# Perform volatility analysis
print("Performing Volatility Analysis...")
volatility_analysis = analyze_volatility_patterns(fx_returns)

# Display results
print("\nVolatility Modeling Results:")
print("=" * 50)

forecast_summary = pd.DataFrame()

for currency, analysis in volatility_analysis.items():
    current_vol = analysis['current_vol']
    vol_clustering = analysis['vol_clustering']
    
    print(f"\n{currency}:")
    print(f"  Current Volatility: {current_vol:.2%}")
    print(f"  Volatility Clustering: {'Yes' if vol_clustering else 'No'}")
    
    if analysis['vol_forecast'] is not None:
        avg_forecast_vol = np.mean(analysis['vol_forecast']) * np.sqrt(252)
        print(f"  30-day Vol Forecast: {avg_forecast_vol:.2%}")
        
        # Store for summary
        forecast_summary.loc[currency, 'Current_Vol'] = current_vol
        forecast_summary.loc[currency, 'Forecast_Vol'] = avg_forecast_vol
        forecast_summary.loc[currency, 'Vol_Change'] = avg_forecast_vol - current_vol
    else:
        print(f"  Model Fitting: Failed")

# Display forecast summary
if not forecast_summary.empty:
    print(f"\nVolatility Forecast Summary:")
    print(forecast_summary.round(4))

# Calculate portfolio-level volatility forecast
print(f"\nPortfolio-Level Volatility Analysis:")

# Current portfolio volatility (from recent returns)
recent_returns = hedge_returns_df['Unhedged (0%)'].tail(30)  # Last 30 days
current_portfolio_vol = recent_returns.std() * np.sqrt(252)

print(f"Current Portfolio Volatility: {current_portfolio_vol:.2%}")

# Forecasted portfolio volatility (weighted by currency exposures)
if not forecast_summary.empty:
    # Exclude USD from non-USD currencies
    non_usd_currencies = [c for c in forecast_summary.index if c != 'USD']
    
    # Weight forecast volatilities by portfolio weights
    forecast_portfolio_vol = 0
    for currency in non_usd_currencies:
        if currency in portfolio_weights and currency in forecast_summary.index:
            weight = portfolio_weights[currency]
            forecast_vol = forecast_summary.loc[currency, 'Forecast_Vol']
            forecast_portfolio_vol += (weight * forecast_vol) ** 2
    
    forecast_portfolio_vol = np.sqrt(forecast_portfolio_vol)
    
    print(f"Forecasted Portfolio Volatility: {forecast_portfolio_vol:.2%}")
    print(f"Expected Volatility Change: {forecast_portfolio_vol - current_portfolio_vol:+.2%}")

print("Volatility Modeling Complete")


In [ ]:
def create_stress_scenarios():
    """
    Create various FX stress scenarios
    """
    scenarios = {}
    
    # Individual currency stress (15% moves)
    for currency in ['EUR', 'GBP', 'JPY', 'EM_BASKET']:
        # Appreciation scenario
        scenarios[f'{currency}_appreciation'] = {
            'name': f'{currency} +15% Appreciation',
            'shocks': {currency: 0.15},
            'description': f'Sudden 15% strengthening of {currency} against USD'
        }
        
        # Depreciation scenario  
        scenarios[f'{currency}_depreciation'] = {
            'name': f'{currency} -15% Depreciation',
            'shocks': {currency: -0.15},
            'description': f'Sudden 15% weakening of {currency} against USD'
        }
    
    # Correlated stress scenarios
    scenarios['developed_appreciation'] = {
        'name': 'Developed Markets +10% Appreciation',
        'shocks': {'EUR': 0.10, 'GBP': 0.10, 'JPY': 0.10},
        'description': 'Simultaneous strengthening of developed market currencies'
    }
    
    scenarios['developed_depreciation'] = {
        'name': 'Developed Markets -10% Depreciation', 
        'shocks': {'EUR': -0.10, 'GBP': -0.10, 'JPY': -0.10},
        'description': 'Simultaneous weakening of developed market currencies'
    }
    
    scenarios['em_crisis'] = {
        'name': 'EM Crisis -25% Depreciation',
        'shocks': {'EM_BASKET': -0.25},
        'description': 'Severe emerging markets currency crisis'
    }
    
    scenarios['usd_weakness'] = {
        'name': 'Broad USD Weakness',
        'shocks': {'EUR': 0.12, 'GBP': 0.15, 'JPY': 0.08, 'EM_BASKET': 0.20},
        'description': 'Broad-based USD weakening across all currencies'
    }
    
    return scenarios

def calculate_stress_impact(scenario_shocks, portfolio_weights, hedge_ratio):
    """
    Calculate portfolio impact from FX stress scenario
    """
    total_impact = 0
    
    for currency, weight in portfolio_weights.items():
        if currency == 'USD':
            continue  # No FX impact for USD assets
            
        # Get FX shock for this currency
        fx_shock = scenario_shocks.get(currency, 0.0)
        
        # Apply hedge ratio (hedging reduces FX impact)
        hedged_fx_shock = fx_shock * (1 - hedge_ratio)
        
        # Portfolio impact = weight × FX shock
        # (Assuming no correlation with local asset returns for stress test)
        currency_impact = weight * hedged_fx_shock
        total_impact += currency_impact
    
    return total_impact

def run_stress_tests(scenarios, portfolio_weights, hedge_ratios, hedge_labels, initial_value):
    """
    Run comprehensive stress testing
    """
    stress_results = pd.DataFrame()
    
    for scenario_key, scenario_data in scenarios.items():
        scenario_name = scenario_data['name']
        shocks = scenario_data['shocks']
        
        row_data = {'Scenario': scenario_name}
        
        for i, hedge_ratio in enumerate(hedge_ratios):
            hedge_label = hedge_labels[i]
            
            # Calculate portfolio impact
            impact = calculate_stress_impact(shocks, portfolio_weights, hedge_ratio)
            
            # Convert to dollar impact
            dollar_impact = initial_value * impact
            
            row_data[f'{hedge_label}_Impact_%'] = impact * 100  # Percentage
            row_data[f'{hedge_label}_Impact_$'] = dollar_impact
        
        stress_results = pd.concat([stress_results, pd.DataFrame([row_data])], ignore_index=True)
    
    return stress_results

# Run stress testing
print("Running Stress Testing...")

stress_scenarios = create_stress_scenarios()
stress_results = run_stress_tests(
    stress_scenarios, 
    portfolio_weights, 
    hedge_ratios, 
    hedge_labels, 
    INITIAL_PORTFOLIO_VALUE
)

# Display stress test results
print("\nStress Test Results:")
print("=" * 80)

# Create summary table for easy viewing
display_cols = ['Scenario', 'Unhedged (0%)_Impact_%', 'Partial Hedge (50%)_Impact_%', 'Full Hedge (100%)_Impact_%']
stress_summary = stress_results[display_cols].copy()
stress_summary.columns = ['Scenario', 'Unhedged (%)', 'Partial Hedge (%)', 'Full Hedge (%)']

print(stress_summary.round(2))

# Identify worst-case scenarios for each hedge strategy
print(f"\nWorst-Case Scenario Analysis:")
print("-" * 50)

for hedge_label in hedge_labels:
    impact_col = f'{hedge_label}_Impact_%'
    worst_scenario_idx = stress_results[impact_col].abs().idxmax()
    worst_scenario = stress_results.loc[worst_scenario_idx]
    
    print(f"\n{hedge_label}:")
    print(f"  Worst Scenario: {worst_scenario['Scenario']}")
    print(f"  Impact: {worst_scenario[impact_col]:.2f}%")
    print(f"  Dollar Loss: ${abs(worst_scenario[f'{hedge_label}_Impact_$']):,.0f}")

# Calculate hedge effectiveness
print(f"\nHedge Effectiveness Analysis:")
print("-" * 40)

hedge_effectiveness = pd.DataFrame()

for _, row in stress_results.iterrows():
    scenario = row['Scenario']
    unhedged_impact = abs(row['Unhedged (0%)_Impact_%'])
    partial_impact = abs(row['Partial Hedge (50%)_Impact_%'])
    full_impact = abs(row['Full Hedge (100%)_Impact_%'])
    
    # Calculate risk reduction
    partial_reduction = (unhedged_impact - partial_impact) / unhedged_impact * 100 if unhedged_impact != 0 else 0
    full_reduction = (unhedged_impact - full_impact) / unhedged_impact * 100 if unhedged_impact != 0 else 0
    
    hedge_effectiveness = pd.concat([
        hedge_effectiveness, 
        pd.DataFrame([{
            'Scenario': scenario,
            'Unhedged_Risk': unhedged_impact,
            '50%_Hedge_Reduction': partial_reduction,
            '100%_Hedge_Reduction': full_reduction
        }])
    ], ignore_index=True)

print(hedge_effectiveness.round(2))

# Calculate average hedge effectiveness
avg_partial_effectiveness = hedge_effectiveness['50%_Hedge_Reduction'].mean()
avg_full_effectiveness = hedge_effectiveness['100%_Hedge_Reduction'].mean()

print(f"\nAverage Hedge Effectiveness:")
print(f"  50% Hedge Strategy: {avg_partial_effectiveness:.1f}% risk reduction")
print(f"  100% Hedge Strategy: {avg_full_effectiveness:.1f}% risk reduction")

print("\nStress Testing Complete")


In [ ]:
def calculate_var_cvar(returns, confidence_levels=[0.95, 0.99], portfolio_value=1):
    """
    Calculate VaR and CVaR using multiple methods
    """
    results = {}
    
    # Remove any NaN values
    returns_clean = returns.dropna()
    
    for confidence_level in confidence_levels:
        alpha = 1 - confidence_level
        
        # Method 1: Historical VaR/CVaR
        historical_var = np.percentile(returns_clean, alpha * 100)
        tail_returns = returns_clean[returns_clean <= historical_var]
        historical_cvar = tail_returns.mean() if len(tail_returns) > 0 else historical_var
        
        # Method 2: Parametric VaR/CVaR (assuming normal distribution)
        mu = returns_clean.mean()
        sigma = returns_clean.std()
        parametric_var = norm.ppf(alpha, mu, sigma)
        
        # CVaR for normal distribution
        parametric_cvar = mu - sigma * norm.pdf(norm.ppf(alpha)) / alpha
        
        # Method 3: Monte Carlo VaR/CVaR
        np.random.seed(42)
        n_simulations = 10000
        simulated_returns = np.random.normal(mu, sigma, n_simulations)
        mc_var = np.percentile(simulated_returns, alpha * 100)
        mc_tail = simulated_returns[simulated_returns <= mc_var]
        mc_cvar = mc_tail.mean() if len(mc_tail) > 0 else mc_var
        
        # Convert to dollar amounts
        results[f'{confidence_level:.0%}'] = {
            'Historical_VaR': historical_var * portfolio_value,
            'Historical_CVaR': historical_cvar * portfolio_value,
            'Parametric_VaR': parametric_var * portfolio_value,
            'Parametric_CVaR': parametric_cvar * portfolio_value,
            'MonteCarlo_VaR': mc_var * portfolio_value,
            'MonteCarlo_CVaR': mc_cvar * portfolio_value,
            'Historical_VaR_%': historical_var * 100,
            'Historical_CVaR_%': historical_cvar * 100,
            'Parametric_VaR_%': parametric_var * 100,
            'Parametric_CVaR_%': parametric_cvar * 100
        }
    
    return results

def calculate_risk_adjusted_metrics(returns, portfolio_value=1):
    """
    Calculate additional risk-adjusted performance metrics
    """
    returns_clean = returns.dropna()
    
    # Annualized metrics
    annual_return = returns_clean.mean() * 252
    annual_volatility = returns_clean.std() * np.sqrt(252)
    sharpe_ratio = annual_return / annual_volatility if annual_volatility > 0 else 0
    
    # Downside metrics
    downside_returns = returns_clean[returns_clean < 0]
    downside_deviation = downside_returns.std() * np.sqrt(252) if len(downside_returns) > 0 else 0
    sortino_ratio = annual_return / downside_deviation if downside_deviation > 0 else 0
    
    # Maximum drawdown
    cumulative_returns = (1 + returns_clean).cumprod()
    rolling_max = cumulative_returns.expanding().max()
    drawdowns = (cumulative_returns - rolling_max) / rolling_max
    max_drawdown = drawdowns.min()
    
    # Hit ratio (percentage of positive days)
    hit_ratio = (returns_clean > 0).mean()
    
    return {
        'Annual_Return_%': annual_return * 100,
        'Annual_Volatility_%': annual_volatility * 100,
        'Sharpe_Ratio': sharpe_ratio,
        'Sortino_Ratio': sortino_ratio,
        'Max_Drawdown_%': max_drawdown * 100,
        'Hit_Ratio_%': hit_ratio * 100,
        'Downside_Deviation_%': downside_deviation * 100
    }

# Calculate VaR and CVaR for each hedge strategy
print("Calculating Risk Metrics...")

confidence_levels = [0.95, 0.99]
risk_metrics_summary = pd.DataFrame()

for i, hedge_ratio in enumerate(hedge_ratios):
    hedge_label = hedge_labels[i]
    returns = hedged_results[hedge_label]['total_return']
    
    print(f"\nAnalyzing {hedge_label}...")
    
    # Calculate VaR/CVaR
    var_cvar_results = calculate_var_cvar(returns, confidence_levels, INITIAL_PORTFOLIO_VALUE)
    
    # Calculate risk-adjusted metrics
    risk_adjusted = calculate_risk_adjusted_metrics(returns, INITIAL_PORTFOLIO_VALUE)
    
    # Combine results
    row_data = {'Strategy': hedge_label}
    row_data.update(risk_adjusted)
    
    # Add VaR/CVaR results
    for conf_level, metrics in var_cvar_results.items():
        for metric_name, value in metrics.items():
            if metric_name.endswith('_%'):
                row_data[f'{conf_level}_{metric_name}'] = value
            elif metric_name.endswith('_VaR') or metric_name.endswith('_CVaR'):
                row_data[f'{conf_level}_{metric_name}'] = value
    
    risk_metrics_summary = pd.concat([risk_metrics_summary, pd.DataFrame([row_data])], ignore_index=True)

# Display comprehensive risk metrics
print("\nComprehensive Risk Metrics Summary:")
print("=" * 80)

# Performance metrics
performance_cols = ['Strategy', 'Annual_Return_%', 'Annual_Volatility_%', 'Sharpe_Ratio', 'Sortino_Ratio', 'Max_Drawdown_%']
print("\nPerformance Metrics:")
print(risk_metrics_summary[performance_cols].round(3))

# VaR metrics (95% confidence)
var_95_cols = ['Strategy', '95%_Historical_VaR_%', '95%_Parametric_VaR_%', '95%_Historical_CVaR_%', '95%_Parametric_CVaR_%']
print(f"\n95% Confidence VaR/CVaR (%):")
print(risk_metrics_summary[var_95_cols].round(3))

# VaR metrics (99% confidence)
var_99_cols = ['Strategy', '99%_Historical_VaR_%', '99%_Parametric_VaR_%', '99%_Historical_CVaR_%', '99%_Parametric_CVaR_%']
print(f"\n99% Confidence VaR/CVaR (%):")
print(risk_metrics_summary[var_99_cols].round(3))

# Dollar VaR comparison
print(f"\nDaily VaR in USD (Historical Method):")
var_comparison = pd.DataFrame()
for _, row in risk_metrics_summary.iterrows():
    var_comparison = pd.concat([
        var_comparison,
        pd.DataFrame([{
            'Strategy': row['Strategy'],
            '95%_VaR_USD': row['95%_Historical_VaR'],
            '99%_VaR_USD': row['99%_Historical_VaR'],
            '95%_CVaR_USD': row['95%_Historical_CVaR'],
            '99%_CVaR_USD': row['99%_Historical_CVaR']
        }])
    ], ignore_index=True)

print(var_comparison.round(0))

# Risk reduction analysis
print(f"\nRisk Reduction from Hedging:")
print("-" * 40)

unhedged_var_95 = abs(risk_metrics_summary.loc[0, '95%_Historical_VaR_%'])
unhedged_var_99 = abs(risk_metrics_summary.loc[0, '99%_Historical_VaR_%'])

for i in range(1, len(hedge_ratios)):
    hedge_label = hedge_labels[i]
    hedged_var_95 = abs(risk_metrics_summary.loc[i, '95%_Historical_VaR_%'])
    hedged_var_99 = abs(risk_metrics_summary.loc[i, '99%_Historical_VaR_%'])
    
    reduction_95 = (unhedged_var_95 - hedged_var_95) / unhedged_var_95 * 100
    reduction_99 = (unhedged_var_99 - hedged_var_99) / unhedged_var_99 * 100
    
    print(f"\n{hedge_label}:")
    print(f"  95% VaR Reduction: {reduction_95:.1f}%")
    print(f"  99% VaR Reduction: {reduction_99:.1f}%")

print("\nRisk Metrics Analysis Complete")


In [ ]:
# Create comprehensive visualizations
fig = plt.figure(figsize=(20, 24))

# 1. FX Rates Evolution
ax1 = plt.subplot(4, 2, 1)
for currency in ['EUR', 'GBP', 'JPY', 'EM_BASKET']:
    normalized_rates = fx_rates[currency] / fx_rates[currency].iloc[0]
    plt.plot(fx_rates.index, normalized_rates, label=currency, linewidth=2)

plt.title('FX Rates Evolution (Normalized to Day 1)', fontsize=14, fontweight='bold')
plt.xlabel('Date')
plt.ylabel('Normalized FX Rate')
plt.legend()
plt.grid(True, alpha=0.3)

# 2. Portfolio Value Evolution
ax2 = plt.subplot(4, 2, 2)
colors = ['#1f77b4', '#ff7f0e', '#2ca02c']
for i, (col, color) in enumerate(zip(['unhedged_value', 'partial_hedge_50pct_value', 'full_hedge_100pct_value'], colors)):
    if col in portfolio_values_all.columns:
        plt.plot(portfolio_values_all.index, portfolio_values_all[col] / 1e6, 
                label=hedge_labels[i], linewidth=2.5, color=color)

plt.title('Portfolio Value Evolution by Hedge Strategy', fontsize=14, fontweight='bold')
plt.xlabel('Date')
plt.ylabel('Portfolio Value ($ Millions)')
plt.legend()
plt.grid(True, alpha=0.3)

# 3. Daily Returns Distribution
ax3 = plt.subplot(4, 2, 3)
colors = ['lightblue', 'orange', 'lightgreen']
for i, hedge_label in enumerate(hedge_labels):
    returns = hedged_results[hedge_label]['total_return'] * 100
    plt.hist(returns, bins=30, alpha=0.6, label=hedge_label, color=colors[i], density=True)

plt.title('Daily Returns Distribution (%)', fontsize=14, fontweight='bold')
plt.xlabel('Daily Return (%)')
plt.ylabel('Density')
plt.legend()
plt.grid(True, alpha=0.3)

# 4. Rolling Volatility
ax4 = plt.subplot(4, 2, 4)
window = 21  # 21-day rolling window
for i, hedge_label in enumerate(hedge_labels):
    returns = hedged_results[hedge_label]['total_return']
    rolling_vol = returns.rolling(window=window).std() * np.sqrt(252) * 100
    plt.plot(rolling_vol.index, rolling_vol, label=hedge_label, linewidth=2)

plt.title(f'{window}-Day Rolling Volatility (%)', fontsize=14, fontweight='bold')
plt.xlabel('Date')
plt.ylabel('Annualized Volatility (%)')
plt.legend()
plt.grid(True, alpha=0.3)

# 5. VaR Comparison
ax5 = plt.subplot(4, 2, 5)
strategies = risk_metrics_summary['Strategy'].tolist()
var_95 = [abs(x) for x in risk_metrics_summary['95%_Historical_VaR_%'].tolist()]
var_99 = [abs(x) for x in risk_metrics_summary['99%_Historical_VaR_%'].tolist()]

x = np.arange(len(strategies))
width = 0.35

plt.bar(x - width/2, var_95, width, label='95% VaR', alpha=0.8, color='lightcoral')
plt.bar(x + width/2, var_99, width, label='99% VaR', alpha=0.8, color='darkred')

plt.title('Value at Risk Comparison (%)', fontsize=14, fontweight='bold')
plt.xlabel('Hedge Strategy')
plt.ylabel('VaR (%)')
plt.xticks(x, strategies, rotation=45)
plt.legend()
plt.grid(True, alpha=0.3)

# 6. Risk-Return Scatter
ax6 = plt.subplot(4, 2, 6)
annual_returns = risk_metrics_summary['Annual_Return_%'].tolist()
annual_vols = risk_metrics_summary['Annual_Volatility_%'].tolist()
colors_scatter = ['red', 'orange', 'green']

for i, strategy in enumerate(strategies):
    plt.scatter(annual_vols[i], annual_returns[i], 
              s=200, label=strategy, color=colors_scatter[i], alpha=0.7)
    plt.annotate(strategy, (annual_vols[i], annual_returns[i]), 
                xytext=(5, 5), textcoords='offset points', fontsize=9)

plt.title('Risk-Return Profile', fontsize=14, fontweight='bold')
plt.xlabel('Annual Volatility (%)')
plt.ylabel('Annual Return (%)')
plt.grid(True, alpha=0.3)

# 7. Currency Contribution Analysis
ax7 = plt.subplot(4, 2, 7)
# Use unhedged returns for currency contribution analysis
contributions = unhedged_returns[[col for col in unhedged_returns.columns if '_contribution' in col]]
contributions.columns = [col.replace('_contribution', '') for col in contributions.columns]

# Calculate cumulative contributions
cumulative_contributions = (1 + contributions).cumprod()

for currency in cumulative_contributions.columns:
    plt.plot(cumulative_contributions.index, cumulative_contributions[currency], 
            label=currency, linewidth=2)

plt.title('Cumulative Currency Contributions (Unhedged)', fontsize=14, fontweight='bold')
plt.xlabel('Date')
plt.ylabel('Cumulative Contribution')
plt.legend()
plt.grid(True, alpha=0.3)

# 8. Drawdown Analysis
ax8 = plt.subplot(4, 2, 8)
for i, hedge_label in enumerate(hedge_labels):
    returns = hedged_results[hedge_label]['total_return']
    cumulative = (1 + returns).cumprod()
    rolling_max = cumulative.expanding().max()
    drawdown = (cumulative - rolling_max) / rolling_max * 100
    
    plt.plot(drawdown.index, drawdown, label=hedge_label, linewidth=2)

plt.title('Portfolio Drawdowns (%)', fontsize=14, fontweight='bold')
plt.xlabel('Date')
plt.ylabel('Drawdown (%)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.fill_between(drawdown.index, drawdown, 0, alpha=0.3)

plt.tight_layout()
plt.show()

# Additional volatility forecast visualization
if volatility_analysis:
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    fig.suptitle('Currency Volatility Analysis', fontsize=16, fontweight='bold')
    
    currencies_plot = ['EUR', 'GBP', 'JPY', 'EM_BASKET']
    
    for i, currency in enumerate(currencies_plot):
        if currency in volatility_analysis:
            ax = axes[i//2, i%2]
            
            # Plot rolling volatility
            rolling_vol = volatility_analysis[currency]['rolling_vol'] * 100
            ax.plot(rolling_vol.index, rolling_vol, label='Historical', linewidth=2, color='blue')
            
            # Plot forecast if available
            if volatility_analysis[currency]['vol_forecast'] is not None:
                forecast_vol = volatility_analysis[currency]['vol_forecast'] * np.sqrt(252) * 100
                forecast_dates = pd.date_range(start=fx_rates.index[-1], periods=31, freq='D')[1:]
                
                if len(forecast_dates) == len(forecast_vol):
                    ax.plot(forecast_dates, forecast_vol, label='30-day Forecast', 
                           linewidth=2, color='red', linestyle='--')
            
            ax.set_title(f'{currency} Volatility (%)', fontweight='bold')
            ax.set_ylabel('Annualized Volatility (%)')
            ax.legend()
            ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

print("All visualizations generated successfully!")


In [ ]:
def generate_strategic_insights(risk_metrics_summary, stress_results, hedge_effectiveness):
    """
    Generate actionable insights based on comprehensive analysis
    """
    insights = []
    
    # Performance Analysis
    best_sharpe_idx = risk_metrics_summary['Sharpe_Ratio'].idxmax()
    best_sharpe_strategy = risk_metrics_summary.loc[best_sharpe_idx, 'Strategy']
    best_sharpe_ratio = risk_metrics_summary.loc[best_sharpe_idx, 'Sharpe_Ratio']
    
    insights.append({
        'category': 'Performance Optimization',
        'insight': f'The {best_sharpe_strategy} demonstrates the highest risk-adjusted returns with a Sharpe ratio of {best_sharpe_ratio:.3f}.',
        'recommendation': 'Consider this strategy for long-term portfolio optimization focusing on risk-adjusted returns.'
    })
    
    # Risk Management
    lowest_var_idx = risk_metrics_summary['95%_Historical_VaR_%'].abs().idxmin()
    lowest_var_strategy = risk_metrics_summary.loc[lowest_var_idx, 'Strategy']
    lowest_var = abs(risk_metrics_summary.loc[lowest_var_idx, '95%_Historical_VaR_%'])
    
    insights.append({
        'category': 'Risk Management',
        'insight': f'The {lowest_var_strategy} provides the lowest downside risk with 95% VaR of {lowest_var:.2f}%.',
        'recommendation': 'Implement this strategy during periods of high market uncertainty or regulatory capital constraints.'
    })
    
    # Stress Testing Insights
    avg_stress_impact = {}
    for strategy in ['Unhedged (0%)', 'Partial Hedge (50%)', 'Full Hedge (100%)']:
        impacts = [abs(row[f'{strategy}_Impact_%']) for _, row in stress_results.iterrows()]
        avg_stress_impact[strategy] = np.mean(impacts)
    
    most_resilient = min(avg_stress_impact, key=avg_stress_impact.get)
    resilience_score = avg_stress_impact[most_resilient]
    
    insights.append({
        'category': 'Stress Resilience',
        'insight': f'Under extreme stress scenarios, {most_resilient} shows the highest resilience with average impact of {resilience_score:.2f}%.',
        'recommendation': 'Deploy this strategy during anticipated periods of FX volatility or geopolitical uncertainty.'
    })
    
    # Cost-Benefit Analysis
    hedging_benefits = hedge_effectiveness['50%_Hedge_Reduction'].mean()
    full_hedging_benefits = hedge_effectiveness['100%_Hedge_Reduction'].mean()
    
    insights.append({
        'category': 'Cost-Benefit Optimization',
        'insight': f'Partial hedging (50%) provides {hedging_benefits:.1f}% risk reduction while full hedging provides {full_hedging_benefits:.1f}%.',
        'recommendation': 'Consider partial hedging as it offers substantial risk reduction with lower hedging costs.'
    })
    
    # Market Regime Analysis
    high_vol_periods = fx_returns.std().nlargest(2).index.tolist()
    low_vol_periods = fx_returns.std().nsmallest(2).index.tolist()
    
    insights.append({
        'category': 'Market Regime Adaptation',
        'insight': f'Currency volatility varies significantly across currencies: {high_vol_periods} show highest volatility.',
        'recommendation': 'Implement dynamic hedge ratios that increase exposure to high-volatility currencies during favorable conditions.'
    })
    
    return insights

def calculate_optimal_hedge_ratio(risk_metrics_summary, target_vol=None, target_return=None):
    """
    Calculate optimal hedge ratio based on specific objectives
    """
    results = {}
    
    # Risk minimization
    min_vol_idx = risk_metrics_summary['Annual_Volatility_%'].idxmin()
    results['risk_minimization'] = {
        'strategy': risk_metrics_summary.loc[min_vol_idx, 'Strategy'],
        'volatility': risk_metrics_summary.loc[min_vol_idx, 'Annual_Volatility_%'],
        'return': risk_metrics_summary.loc[min_vol_idx, 'Annual_Return_%']
    }
    
    # Return maximization
    max_return_idx = risk_metrics_summary['Annual_Return_%'].idxmax()
    results['return_maximization'] = {
        'strategy': risk_metrics_summary.loc[max_return_idx, 'Strategy'],
        'volatility': risk_metrics_summary.loc[max_return_idx, 'Annual_Volatility_%'],
        'return': risk_metrics_summary.loc[max_return_idx, 'Annual_Return_%']
    }
    
    # Sharpe maximization
    max_sharpe_idx = risk_metrics_summary['Sharpe_Ratio'].idxmax()
    results['sharpe_maximization'] = {
        'strategy': risk_metrics_summary.loc[max_sharpe_idx, 'Strategy'],
        'volatility': risk_metrics_summary.loc[max_sharpe_idx, 'Annual_Volatility_%'],
        'return': risk_metrics_summary.loc[max_sharpe_idx, 'Annual_Return_%'],
        'sharpe': risk_metrics_summary.loc[max_sharpe_idx, 'Sharpe_Ratio']
    }
    
    return results

# Generate insights
print("Generating Strategic Insights...")
strategic_insights = generate_strategic_insights(risk_metrics_summary, stress_results, hedge_effectiveness)

print("\n" + "="*80)
print("STRATEGIC INSIGHTS AND RECOMMENDATIONS")
print("="*80)

for i, insight in enumerate(strategic_insights, 1):
    print(f"\n{i}. {insight['category'].upper()}")
    print("-" * 50)
    print(f"Analysis: {insight['insight']}")
    print(f"Recommendation: {insight['recommendation']}")

# Calculate optimal strategies
optimal_strategies = calculate_optimal_hedge_ratio(risk_metrics_summary)

print(f"\n" + "="*80)
print("OPTIMAL HEDGE RATIO RECOMMENDATIONS")
print("="*80)

for objective, details in optimal_strategies.items():
    print(f"\n{objective.replace('_', ' ').title()}:")
    print(f"   Recommended Strategy: {details['strategy']}")
    print(f"   Expected Return: {details['return']:.2f}%")
    print(f"   Expected Volatility: {details['volatility']:.2f}%")
    if 'sharpe' in details:
        print(f"   Sharpe Ratio: {details['sharpe']:.3f}")

# Implementation roadmap
print(f"\n" + "="*80)
print("IMPLEMENTATION ROADMAP")
print("="*80)

implementation_steps = [
    {
        'phase': 'Phase 1: Foundation (Weeks 1-2)',
        'actions': [
            'Establish FX data feeds and risk measurement infrastructure',
            'Implement basic VaR/CVaR calculation capabilities',
            'Set up portfolio exposure tracking by currency'
        ]
    },
    {
        'phase': 'Phase 2: Strategy Development (Weeks 3-4)',
        'actions': [
            'Develop dynamic hedge ratio adjustment mechanisms',
            'Implement stress testing framework',
            'Create real-time volatility monitoring'
        ]
    },
    {
        'phase': 'Phase 3: Optimization (Weeks 5-6)',
        'actions': [
            'Deploy GARCH-based volatility forecasting',
            'Integrate market regime detection',
            'Establish automated hedge ratio recommendations'
        ]
    },
    {
        'phase': 'Phase 4: Monitoring (Ongoing)',
        'actions': [
            'Daily risk metric calculation and reporting',
            'Monthly strategy performance review',
            'Quarterly model validation and recalibration'
        ]
    }
]

for step in implementation_steps:
    print(f"\n{step['phase']}")
    for action in step['actions']:
        print(f"   • {action}")

# Key Performance Indicators
print(f"\n" + "="*80)
print("KEY PERFORMANCE INDICATORS (KPIs)")
print("="*80)

kpis = [
    {'metric': 'Portfolio Volatility', 'target': '< 15% annually', 'frequency': 'Daily'},
    {'metric': '95% VaR', 'target': '< 2.5% daily', 'frequency': 'Daily'},
    {'metric': 'Sharpe Ratio', 'target': '> 0.8', 'frequency': 'Monthly'},
    {'metric': 'Maximum Drawdown', 'target': '< 8%', 'frequency': 'Monthly'},
    {'metric': 'Hedge Effectiveness', 'target': '> 70% in stress scenarios', 'frequency': 'Quarterly'},
    {'metric': 'Model Accuracy', 'target': '> 85% forecast accuracy', 'frequency': 'Quarterly'}
]

print(f"\n{'Metric':<20} {'Target':<25} {'Frequency':<15}")
print("-" * 60)
for kpi in kpis:
    print(f"{kpi['metric']:<20} {kpi['target']:<25} {kpi['frequency']:<15}")

# Risk warnings and considerations
print(f"\n" + "="*80)
print("RISK WARNINGS AND CONSIDERATIONS")
print("="*80)

warnings = [
    "Model Risk: GARCH and ARIMA models may not capture extreme market events",
    "Hedging Costs: Transaction costs and bid-ask spreads can erode hedging benefits",
    "Basis Risk: Imperfect correlation between hedge instruments and portfolio exposures",
    "Liquidity Risk: FX hedging instruments may become illiquid during stress periods",
    "Regulatory Risk: Changes in derivatives regulations may impact hedging strategies"
]

for i, warning in enumerate(warnings, 1):
    print(f"\n{i}. {warning}")

print(f"\n" + "="*80)
print("ANALYSIS COMPLETE - READY FOR IMPLEMENTATION")
print("="*80)

# Summary statistics for final review
final_summary = {
    'simulation_days': SIMULATION_DAYS,
    'portfolio_value': f"${INITIAL_PORTFOLIO_VALUE:,.0f}",
    'currencies_analyzed': len(currencies),
    'hedge_strategies': len(hedge_ratios),
    'stress_scenarios': len(stress_scenarios),
    'best_strategy': optimal_strategies['sharpe_maximization']['strategy'],
    'risk_reduction': f"{hedge_effectiveness['50%_Hedge_Reduction'].mean():.1f}%"
}

print(f"\nFinal Summary:")
for key, value in final_summary.items():
    print(f"   {key.replace('_', ' ').title()}: {value}")

print(f"\nCurrency Overlay Simulation Engine Complete!")
print(f"Ready for production deployment and real-time risk management.")
